# Risky Bank Loans - _Decision Trees_

## AnacondaCon 2018

#### Paige Bailey - 10 April 2018

-------------------

## Step 0: Install and Load Required Packages

In [1]:
install.packages('C50')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [2]:
install.packages('gmodels')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [1]:
library(C50)
library(gmodels)

## Step 1: Collect the Data

In [2]:
# Shamelessly swiped the .csv from UC Irvine
# But it's okay, because everyone uses the German credit agency data set
# http://archive.ics.uci.edu/ml/

credit <- read.csv("credit.csv")
str(credit)

# You can see that the factors are a mix of numeric and categorical features.

'data.frame':	1000 obs. of  17 variables:
 $ checking_balance    : Factor w/ 4 levels "< 0 DM","> 200 DM",..: 1 3 4 1 1 4 4 3 4 3 ...
 $ months_loan_duration: int  6 48 12 42 24 36 24 36 12 30 ...
 $ credit_history      : Factor w/ 5 levels "critical","good",..: 1 2 1 2 4 2 2 2 2 1 ...
 $ purpose             : Factor w/ 6 levels "business","car",..: 5 5 4 5 2 4 5 2 5 2 ...
 $ amount              : int  1169 5951 2096 7882 4870 9055 2835 6948 3059 5234 ...
 $ savings_balance     : Factor w/ 5 levels "< 100 DM","> 1000 DM",..: 5 1 1 1 1 5 4 1 2 1 ...
 $ employment_duration : Factor w/ 5 levels "< 1 year","> 7 years",..: 2 3 4 4 3 3 2 3 4 5 ...
 $ percent_of_income   : int  4 2 2 2 3 2 3 2 2 4 ...
 $ years_at_residence  : int  4 2 3 4 4 4 4 2 4 2 ...
 $ age                 : int  67 22 49 45 53 35 53 35 61 28 ...
 $ other_credit        : Factor w/ 3 levels "bank","none",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ housing             : Factor w/ 3 levels "other","own",..: 2 2 2 1 1 1 2 3 2 2 ...
 $ exi

In [4]:
# Take a closer look at checking and savings account balances - categorical
# Note: all values are in marks
plot(credit$savings_balance)
table(credit$checking_balance)

table(credit$savings_balance)

# Loan features - numeric
summary(credit$months_loan_duration)

summary(credit$amount)

# Our predictor - whether the individual defaulted on a loan
table(credit$default)


    < 0 DM   > 200 DM 1 - 200 DM    unknown 
       274         63        269        394 


     < 100 DM     > 1000 DM  100 - 500 DM 500 - 1000 DM       unknown 
          603            48           103            63           183 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    4.0    12.0    18.0    20.9    24.0    72.0 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    250    1366    2320    3271    3972   18424 


 no yes 
700 300 

In [7]:
set.seed(12345)
# Since values are not sorted in random order, we have to randomly select.
# runif() selects from a uniform distribution, and order() orders it.
credit_rand <- credit[order(runif(1000)), ]

# What is the order() function?
order(c(0.5, 0.25, 0.75, 0.1))

[1] 4 2 1 3

In [8]:
# Check to make sure you have the same data frame, just sorted differently.
summary(credit$amount)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    250    1366    2320    3271    3972   18424 

In [9]:
summary(credit_rand$amount)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    250    1366    2320    3271    3972   18424 

In [10]:
# Check the first few values in each data frame.
head(credit$amount)

[1] 1169 5951 2096 7882 4870 9055

In [11]:
head(credit_rand$amount)

[1] 1199 2576 1103 4020 1501 1568

In [12]:
# 90% for training, and 10% for testing.
credit_train <- credit_rand[1:900, ]
credit_test  <- credit_rand[901:1000, ]

# Check to make sure you have ~30% of defaulted loans in each of the data sets.
prop.table(table(credit_train$default))


       no       yes 
0.7022222 0.2977778 

In [13]:
prop.table(table(credit_test$default))


  no  yes 
0.68 0.32 

## Step 3: Train a Model on the Data

In [14]:
# 17th column is the default class variable
credit_model <- C5.0(credit_train[-17], credit_train$default)

# Learn more about the tree.
credit_model


Call:
C5.0.default(x = credit_train[-17], y = credit_train$default)

Classification Tree
Number of samples: 900 
Number of predictors: 16 

Tree size: 67 

Non-standard options: attempt to group attributes


In [15]:
summary(credit_model)


Call:
C5.0.default(x = credit_train[-17], y = credit_train$default)


C5.0 [Release 2.07 GPL Edition]  	Fri Feb 23 17:50:15 2018
-------------------------------

Class specified by attribute `outcome'

Read 900 cases (17 attributes) from undefined.data

Decision tree:

checking_balance = unknown: no (358/44)
checking_balance in {< 0 DM,> 200 DM,1 - 200 DM}:
:...credit_history in {perfect,very good}:
    :...dependents > 1: yes (10/1)
    :   dependents <= 1:
    :   :...savings_balance = < 100 DM: yes (39/11)
    :       savings_balance in {> 1000 DM,500 - 1000 DM,unknown}: no (8/1)
    :       savings_balance = 100 - 500 DM:
    :       :...checking_balance = < 0 DM: no (1)
    :           checking_balance in {> 200 DM,1 - 200 DM}: yes (5/1)
    credit_history in {critical,good,poor}:
    :...months_loan_duration <= 11: no (87/14)
        months_loan_duration > 11:
        :...savings_balance = > 1000 DM: no (13)
            savings_balance in {< 100 DM,100 - 500 DM,500 - 1000 DM,unk

## Step 4: Evaluate Model Performance

In [15]:
# Apply decision tree to test data set.
credit_pred <- predict(credit_model, credit_test)

# Compare predicted class values to actuals
# prop.c = column percentages; prop.r = row percentages
CrossTable(credit_test$default, credit_pred,
             prop.chisq = FALSE, prop.c = FALSE, prop.r = FALSE,
             dnn = c('actual default', 'predicted default'))


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
               | predicted default 
actual default |        no |       yes | Row Total | 
---------------|-----------|-----------|-----------|
            no |        57 |        11 |        68 | 
               |     0.570 |     0.110 |           | 
---------------|-----------|-----------|-----------|
           yes |        16 |        16 |        32 | 
               |     0.160 |     0.160 |           | 
---------------|-----------|-----------|-----------|
  Column Total |        73 |        27 |       100 | 
---------------|-----------|-----------|-----------|

 


## Step 5: Improve Model Performance

In [16]:
# ADAPTIVE BOOSTING = process in which many decision trees are built, and trees vote on the
# best class for each sample. Combine a number of weak performers, and you have a team that 
# is stronger than any one of the learners alone.

# If you're interested in literature: The idea of boosting is based largely upon research by Rob Schapire 
# and Yoav Freund. For more information, try searching the web for their publications or their recent 
# textbook: "Boosting: Foundations and Algorithms Understanding Rule Learners" (The MIT Press, 2012).

credit_boost10 <- C5.0(credit_train[-17], credit_train$default,
                         trials = 10)
credit_boost10


Call:
C5.0.default(x = credit_train[-17], y = credit_train$default, trials = 10)

Classification Tree
Number of samples: 900 
Number of predictors: 16 

Number of boosting iterations: 10 
Average tree size: 56 

Non-standard options: attempt to group attributes


In [17]:
summary(credit_boost10)


Call:
C5.0.default(x = credit_train[-17], y = credit_train$default, trials = 10)


C5.0 [Release 2.07 GPL Edition]  	Wed Feb 21 08:20:25 2018
-------------------------------

Class specified by attribute `outcome'

Read 900 cases (17 attributes) from undefined.data

-----  Trial 0:  -----

Decision tree:

checking_balance = unknown: no (358/44)
checking_balance in {< 0 DM,> 200 DM,1 - 200 DM}:
:...credit_history in {perfect,very good}:
    :...dependents > 1: yes (10/1)
    :   dependents <= 1:
    :   :...savings_balance = < 100 DM: yes (39/11)
    :       savings_balance in {> 1000 DM,500 - 1000 DM,unknown}: no (8/1)
    :       savings_balance = 100 - 500 DM:
    :       :...checking_balance = < 0 DM: no (1)
    :           checking_balance in {> 200 DM,1 - 200 DM}: yes (5/1)
    credit_history in {critical,good,poor}:
    :...months_loan_duration <= 11: no (87/14)
        months_loan_duration > 11:
        :...savings_balance = > 1000 DM: no (13)
            savings_balance in {< 

In [18]:
credit_boost_pred10 <- predict(credit_boost10, credit_test)
CrossTable(credit_test$default, credit_boost_pred10,
             prop.chisq = FALSE, prop.c = FALSE, prop.r = FALSE,
             dnn = c('actual default', 'predicted default'))


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
               | predicted default 
actual default |        no |       yes | Row Total | 
---------------|-----------|-----------|-----------|
            no |        60 |         8 |        68 | 
               |     0.600 |     0.080 |           | 
---------------|-----------|-----------|-----------|
           yes |        15 |        17 |        32 | 
               |     0.150 |     0.170 |           | 
---------------|-----------|-----------|-----------|
  Column Total |        75 |        25 |       100 | 
---------------|-----------|-----------|-----------|

 


In [19]:
# Apply a penalty to different types of errors, because
# some mistakes are more costly than others.
error_cost <- matrix(c(0, 1, 4, 0), nrow = 2)
error_cost

0,4
1,0


In [20]:
credit_cost <- C5.0(credit_train[-17], credit_train$default,
                            costs = error_cost)

Warning message:
“no dimnames were given for the cost matrix; the factor levels will be used”

In [21]:
credit_cost_pred <- predict(credit_cost, credit_test)

# See if the confusion matrix changes, based on the weighting.
CrossTable(credit_test$default, credit_cost_pred,
             prop.chisq = FALSE, prop.c = FALSE, prop.r = FALSE,
             dnn = c('actual default', 'predicted default'))


 
   Cell Contents
|-------------------------|
|                       N |
|         N / Table Total |
|-------------------------|

 
Total Observations in Table:  100 

 
               | predicted default 
actual default |        no |       yes | Row Total | 
---------------|-----------|-----------|-----------|
            no |        42 |        26 |        68 | 
               |     0.420 |     0.260 |           | 
---------------|-----------|-----------|-----------|
           yes |         6 |        26 |        32 | 
               |     0.060 |     0.260 |           | 
---------------|-----------|-----------|-----------|
  Column Total |        48 |        52 |       100 | 
---------------|-----------|-----------|-----------|

 
